In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: presto.

In [2]:
from xtalpaint.aiida.workgraphs.inpainting import setup_inpainting_wg
from xtalpaint.inpainting.config_schema import InpaintingWorkGraphConfig
from xtalpaint.aiida.data import BatchedStructures
from ase.io import read
from pymatgen.io.ase import AseAtomsAdaptor

MODELS_PROJECT_ROOT: /home/reents_t/project/dev-mattergen-inpainting/git/mattergen/mattergen


In [3]:
structures = read("test-structures.extxyz", index=':')
structures = {
    a.info['uuid']: AseAtomsAdaptor.get_structure(a) for a in structures
}

In [4]:
len(structures)

5

In [5]:
param_grid = {
    "N_steps": 5,
    "coordinates_snr": 0.2,
    "n_corrector_steps": 1,
    "batch_size": 1000,
}

In [6]:
ENV_ACTIVATION_CMD = "source ~/.aiida_venvs/dev-mattergen-inpainting/bin/activate"

inputs = InpaintingWorkGraphConfig(
    inpainting_pipeline_params={
        "predictor_corrector": "baseline",
        "inpainting_model_params": param_grid,
        "pretrained_name": "mattergen_base",
        "sampling_config_path": "/home/reents_t/project/dev-mattergen-inpainting/git/mattergen/sampling_conf",
    },
    structures=BatchedStructures(
        {k.replace("-", "_"): s for k, s in structures.items()}
    ),
    gen_inpainting_candidates_params={
        "n_inp": {
            k.replace("-", "_"): int(s.composition["H"])
            for k, s in structures.items()
        },
        "element": "H",
        "num_samples": 1,
    },
    relax=True,
    full_relax=True,
    full_relax_wo_pre_relax=False,
    run_inpainting=True,
    relax_kwargs={
        "elements_to_relax": ["H"],
        "fmax": 0.01,
        "max_natoms_per_batch": 5000,
        "load_path": "MatterSim-v1.0.0-5M.pth",
        "max_n_steps": 50,
        "device": "cuda",
        "mlip": "mattersim",
        "optimizer": "BFGS",
        "return_initial_energies": False,
        "return_initial_forces": False,
        "return_final_forces": False,
    },
    gen_inpainting_candidates_options={
        "custom_scheduler_commands": f"{ENV_ACTIVATION_CMD}\nexport PYTHONBREAKPOINT=0",
    },
    options={
        "prepend_text": f"{ENV_ACTIVATION_CMD}\nexport PYTHONBREAKPOINT=0",
    },
    evaluate_params={"max_workers": 5, "metrics": ["match", "rmsd"]},
    evaluate=True,
)

In [7]:
wg = setup_inpainting_wg(
    inputs
)

In [8]:
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [9]:
wg.run()

01/07/2026 03:51:49 PM <183876> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.9.27 is not supported and will cause unexpected problems!
01/07/2026 03:51:49 PM <183876> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
01/07/2026 03:51:49 PM <183876> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
01/07/2026 03:51:59 PM <183876> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [250993|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_inpainting_candidates
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[250993|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_inpainting_candidates
01/07/2026 03:52:02 PM <183876> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [250993|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 250997
REPORT:aiida.orm.nodes.